In [53]:
!pip install tensorboard==2.2.0 --use-feature=2020-resolver
!pip install tensorflow_text>=2.0.0rc0 --use-feature=2020-resolver
!pip install -U sentence-transformers --use-feature=2020-resolver

     |████████████████████████████████| 2.8 MB 3.0 MB/s eta 0:00:01
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
ERROR: tensorflow 2.3.0 requires tensorboard<3,>=2.3.0, but you'll have tensorboard 2.2.0 which is incompatible.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
ERROR: pytorch-lightning 0.9.0 requires tensorboard==2.2.0, but you'll have tensorboard 2.3.0 which is incompatible.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [54]:
import json
import numpy as np
import pandas as pd
import re
import tensorflow_hub as hub
import tensorflow_text
import torch
from sentence_transformers import SentenceTransformer, util, models

In [55]:
with open('../input/queries/queries.json') as json_file:
    data = json.load(json_file)
data

{'Names': [{'Name': 'Хоз-питьевой водопровод В1',
   'Items': [{'Item': 'Фланец плоский стальной свободный DN150 PN1,0 МПа',
     'Model': '',
     'Manufacturer': '',
     'Quantity': '11',
     'Measure': 'шт.'},
    {'Item': 'Упор бетонный',
     'Model': '',
     'Manufacturer': '',
     'Quantity': '3',
     'Measure': 'шт.'},
    {'Item': 'Гидрант пожарный подземный Н-1,5',
     'Model': '',
     'Manufacturer': '',
     'Quantity': '3',
     'Measure': 'шт.'}]},
  {'Name': 'Водопровод хозяйственно-питьевой В1',
   'Items': [{'Item': 'Задвижка AVK фланцевая короткая DN80 с ответными фланцами и крепежом',
     'Model': '',
     'Manufacturer': '',
     'Quantity': '2',
     'Measure': 'шт.'},
    {'Item': 'Труба ПЭ 100 SDR 13,6-560х41,2 техническая',
     'Model': '',
     'Manufacturer': '',
     'Quantity': '42,0',
     'Measure': 'м'},
    {'Item': 'Теплоизоляция ROCKWOOL TEX MAT 100 мм, с уплот. по толщ. 1, 2',
     'Model': '',
     'Manufacturer': '',
     'Quantity': '10,5'

In [56]:
queries = [item['Item'] for project in data['Names'] for item in project['Items']]

In [57]:
handbook = pd.read_excel('../input/handbook/handbook.xls')
handbook.head()

,Код,Артикул,Наименование,Полное наименование,Вид номенклатуры,Базовая единица измерения,Единица хранения остатков,Номер ГТД,Страна происхождения,Ставка НДС,Номенклатурная группа,Ценовая группа,Комментарий
0,1920,NaN,"500х23,9 труба ПНД ""техническая""","Труба ПНД ""техническая"" 500х23,9",Товар,м,м,NaN,NaN,20%,NaN,NaN,NaN
1,УТ000004069,NaN,003L0280 RLV-K R1/2-G3/4 Узел подключения для ...,003L0280 RLV-K R1/2-G3/4 Узел подключения для ...,Товар,шт,шт,NaN,NaN,20%,NaN,NaN,NaN
2,УТ000000195,NaN,003L0280 Клапан RLV-K G 3/4A; G 1/2А,"003L0280 Н-образный клапан RLV-K, прямой, для ...",Товар,шт,шт,NaN,NaN,20%,NaN,NaN,NaN
3,УТ000000198,NaN,003L7603 Регулятор перепада давления ASV-PV Ду25,003L7603 Регулятор перепада давления ASV-PV с ...,Товар,шт,шт,NaN,NaN,20%,NaN,NaN,NaN
4,УТ000001434,NaN,003L7604 Регулятор перепада давления ASV-PV Ду40,003L7604 Регулятор перепада давления ASV-PV с ...,Товар,шт,шт,NaN,NaN,20%,NaN,NaN,NaN


In [58]:
# 2 или 3
print(handbook.iloc[:, 2].isnull().values.any())
print(handbook.iloc[:, 3].isnull().values.any())

False
True


In [59]:
handbook.iloc[:, 2] = handbook.iloc[:, 2].fillna('')

In [60]:
handbook.duplicated().any()

False

In [61]:
# Corpus with example sentences
handbook_names = handbook.iloc[:, 2].tolist()

In [62]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5

## Multilingual BERT for sentences

In [63]:
# # Use BERT for mapping tokens to embeddings
# word_embedding_model = models.BERT('distilbert-base-multilingual-cased')

# # Apply mean pooling to get one fixed sized sentence vector
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
#                                pooling_mode_mean_tokens=True,
#                                pooling_mode_cls_token=False,
#                                pooling_mode_max_tokens=False)

# bert_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [64]:
# handbook_bert_embeddings = bert_model.encode(handbook_names, convert_to_tensor=True)

In [65]:
# for query in queries:
#     query_embedding = bert_model.encode(query, convert_to_tensor=True)
#     cos_scores = util.pytorch_cos_sim(query_embedding, handbook_bert_embeddings)[0]
#     cos_scores = cos_scores.cpu()
    
#     top_results = torch.topk(cos_scores, k=top_k)
    
#     print("Query:", query)
#     print("\nTop 5 most similar sentences:")
#     # print(handbook.iloc[top_results.indices])
#     for score, idx in zip(top_results[0], top_results[1]):
#         print(handbook_names[idx], "(Score: %.4f)" % (score))

## Universal Sentence Encoder Multilingual

In [66]:
# use_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

# # Compute embeddings.
# use_embeddings = use_embed(handbook_names)

In [67]:
# # Compute similarity matrix. Higher score indicates greater similarity.

# # inner only for normalized vectors
# # convert_to_tensor
# for query in queries:
    
#     query_embedding = use_embed(query)
#     similarity_matrix = np.inner(use_embeddings, query_embedding)
#     top_results = (-similarity_matrix).argsort(axis=0)[:top_k].ravel()
    
#     print('\nQuery:', query)
#     print('\nTop 5 most similar sentences:')
#     # print(handbook.iloc[top_results])
#     for idx in top_results:
#         print(handbook_names[idx], '(Score: %.4f)' % (similarity_matrix[idx]))

In [68]:
# specifications = []
# for n in range(1, 15):
#     with open(f'../input/specifications-examples/{n}.txt', 'r') as file:
#         items = file.readlines()
#         for item in items:
#             if re.findall(r'^\d+[\.]? \|', item):
#                 positions = [character.start() for character in re.finditer('\|', item)]
#                 if (len(positions) > 3) and (len(positions) < 8):
#                     specifications.append(item[positions[0] + 2: positions[1] - 1].strip(' -'))
# specifications = [specification for specification in specifications if specification]

In [69]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar
!conda install -y gdown 
!gdown https://drive.google.com/uc?id=1U4TT9cGtdYL8I0G5NSd2gCvia9wpFpDr

--2020-11-09 11:09:27--  https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26634240 (25M) [application/x-tar]
Saving to: ‘navec_news_v1_1B_250K_300d_100q.tar.1’

navec_news_v1_1B_25 100%[===================>]  25.40M  9.44MB/s    in 2.7s    

2020-11-09 11:09:30 (9.44 MB/s) - ‘navec_news_v1_1B_250K_300d_100q.tar.1’ saved [26634240/26634240]

Solving environment: done

# All requested packages already installed.

Downloading...
From: https://drive.google.com/uc?id=1U4TT9cGtdYL8I0G5NSd2gCvia9wpFpDr
To: /kaggle/working/slovnet_morph_news_v1.tar
2.58MB [00:00, 113MB/s]


In [70]:
!pip install razdel
!pip install navec
!pip install slovnet

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [71]:
import gdown
url = 'https://drive.google.com/file/d/1EGGLa7YJo93hG7dEAdV6TEPWMq3DqIh2/view?usp=sharing'
output = 'slovnet_morph_news_v1.tar.gz'
gdown.download(url, output, quiet=False)

/opt/conda/lib/python3.7/site-packages/gdown/parse_url.py:31: UserWarning: You specified Google Drive Link but it is not the correct link to download the file. Maybe you should try: https://drive.google.com/uc?id=1EGGLa7YJo93hG7dEAdV6TEPWMq3DqIh2
  url="https://drive.google.com/uc?id={}".format(file_id)
Downloading...
From: https://drive.google.com/file/d/1EGGLa7YJo93hG7dEAdV6TEPWMq3DqIh2/view?usp=sharing
To: /kaggle/working/slovnet_morph_news_v1.tar.gz

70.8kB [00:00, 6.27MB/s]


'slovnet_morph_news_v1.tar.gz'

In [72]:
from razdel import sentenize, tokenize
from navec import Navec
from slovnet import Morph

In [73]:
navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
morph = Morph.load('slovnet_morph_news_v1.tar', batch_size=4)
morph.navec(navec)

Morph(
    infer=MorphInfer(
        model=Morph(
            emb=WordShapeEmbedding(
                word=NavecEmbedding(
                    id='news_v1_1B_250K_300d_100q',
                    indexes=Weight(
                        shape=[250002,
                         100],
                        dtype='uint8',
                        array=array([[176, 222, 248, ..., 244, 183, 191],
                               [215, 200, 168, ..., 120, 217,  21],
                               [ 83, 174,  54, ..., 106,  88, 251],
                               ...,
                               [133, 125, 123, ..., 124,  94,  24],
                               [183,  49, 180, ..., 151, 167,  68],
                               [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)
                    ),
                    codes=Weight(
                        shape=[100,
                         256,
                         3],
                        dtype='float32',
                       

In [74]:
def element_extraction(text):
    digit_terms = re.findall('\w*\d*[\.,]?\d+\w*', text)

    chunk = []
    for sentence in sentenize(text.lower()):
        print(sentence)
        tokens = [_.text for _ in tokenize(sentence.text)]
        chunk.append(tokens)
    markup = next(morph.map(chunk))
    
    nouns = []
    adjectives = []
    for token in markup.tokens:
        if (token.pos == 'NOUN' or token.pos == 'PNOUN') and not(re.findall('\d+', token.text)):
            nouns.append(token.text)
        if token.pos == 'ADJ' or token.pos == 'ADV':
            adjectives.append(token.text)
    return digit_terms, nouns, adjectives

## RuBERT

In [75]:
from sentence_transformers import SentenceTransformer, models

word_embedding_model = models.Transformer('../input/pytorch-rubert/', max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

rubert_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [76]:
handbook_rubert_embeddings = rubert_model.encode(handbook_names, convert_to_tensor=True)

In [95]:
results = {'Исходный текст': [],
           'Количество': [],
           'Единицы измерения': [],
           'Предлагаемый вариант': [],
           'Коэффициент сходства, %': []}

In [78]:
def improved_cosine_similarity(record, addendums_coef, adj_coef):
    digit_terms, nouns, adjectives = element_extraction(record)
    digit_terms_encoded = rubert_model.encode(' '.join(digit_terms), convert_to_tensor=True)
    nouns_encoded = rubert_model.encode(' '.join(nouns), convert_to_tensor=True)
    adjectives_encoded = rubert_model.encode(' '.join(adjectives), convert_to_tensor=True)
    
    digits_cos_scores = util.pytorch_cos_sim(query_digit_terms_encoded, digit_terms_encoded).cpu()
    nouns_cos_scores = util.pytorch_cos_sim(query_nouns_encoded, nouns_encoded).cpu()
    adj_cos_scores = util.pytorch_cos_sim(query_adjectives_encoded, adjectives_encoded).cpu()
    
    return addendums_coef * (digits_cos_scores + nouns_cos_scores + adj_coef * adj_cos_scores)

In [98]:
# попробовать с нулем
# попробовать без плюса
top_k = 20
addendums_coef = 0.1
adj_coef = 0.8
threshold = 0.92

for query in queries:
    query_embedding = rubert_model.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, handbook_rubert_embeddings)[0] # change to rubert
    cos_scores = cos_scores.cpu()
    
    top_results = torch.topk(cos_scores, k=top_k)
    top_results_values = top_results.values.numpy()
    top_results_indices = top_results.indices.numpy()

    query_digit_terms, query_nouns, query_adjectives = element_extraction(query)
    query_digit_terms_encoded = rubert_model.encode(' '.join(query_digit_terms), convert_to_tensor=True)
    query_nouns_encoded = rubert_model.encode(' '.join(query_nouns), convert_to_tensor=True)
    query_adjectives_encoded = rubert_model.encode(' '.join(query_adjectives), convert_to_tensor=True)
         
    top_results_values = [top_results_values[n] + improved_cosine_similarity(handbook_names[top_results_indices[n]],
                                                                             addendums_coef,
                                                                             adj_coef) for n in range(top_k)]
    max_cos_indices = np.argsort(top_results_values)[::-1]

    new_top_results_values = [top_results_values[n] for n in max_cos_indices][:1:]
    top_results_indices = [top_results_indices[n] for n in max_cos_indices][:1:]
    self_similarity = 1 + addendums_coef * (2 + adj_coef)
    
    similarity_coef = float(new_top_results_values[0][0][0]) / self_similarity

#     for score, index in zip(new_top_results_values, top_results_indices):
#         print(handbook_names[index], "(Score: %.4f)" % (score))

    if similarity_coef >= threshold:
        results['Исходный текст'].append(query)
        results['Количество'].append(1)
        results['Единицы измерения'].append('шт')
        results['Предлагаемый вариант'].append(handbook_names[top_results_indices[0]])
        results['Коэффициент сходства, %'].append(similarity_coef * 100)
    else:
        results['Исходный текст'].append(query)
        results['Количество'].append(1)
        results['Единицы измерения'].append('шт')
        results['Предлагаемый вариант'].append('')
        results['Коэффициент сходства, %'].append('')    

# without other information


Substring(0, 49, 'фланец плоский стальной свободный dn150 pn1,0 мпа')



Substring(0, 40, 'фланец плоский стальной ду80 (90тр) pn16')



Substring(0, 33, 'фланец плоский стальной ду65 pn16')



Substring(0, 23, 'тройник стальной 57х3-4')



Substring(0, 34, 'переход 45х3-38х2 (40х32) стальной')


...


Substring(0, 22, 'внутренняя канализация')



Substring(0, 34, 'pipe life - внутренняя канализация')



Substring(0, 29, 'корсис - наружняя канализация')



Substring(0, 49, 'труба 110х2,7х1000 пп канализационная с раструбом')



Substring(0, 21, 'ремонт стойки портала')


In [100]:
results = pd.DataFrame(results)
results

,Исходный текст,Количество,Единицы измерения,Предлагаемый вариант,"Коэффициент сходства, %"
0,"Фланец плоский стальной свободный DN150 PN1,0 МПа",1,шт,Фланец плоский стальной Ду80 (90тр) PN16,91.8683
1,Упор бетонный,1,шт,Упор бетонный УГ1,92.7666
2,"Гидрант пожарный подземный Н-1,5",1,шт,"Гидрант пожарный Н 1,5 м JAFAR",95.156
3,Задвижка AVK фланцевая короткая DN80 с ответны...,1,шт,"Задвижка клиновая, фланцевая, короткая PN 16, ...",92.5341
4,"Труба ПЭ 100 SDR 13,6-560х41,2 техническая",1,шт,"140х8,3 труба SDR17 ""техническая"" отр.13м",92.281
5,"Теплоизоляция ROCKWOOL TEX MAT 100 мм, с уплот...",1,шт,,
6,Фильтрующий модуль для очистки сточных вод,1,шт,,
7,"Фланец плоский стальной свободный DN150 PN1,0 МПа",1,шт,,
8,Упор бетонный,1,шт,Упор бетонный УГ1,92.7666
9,"Гидрант пожарный подземный Н-1,5",1,шт,"Гидрант пожарный Н 1,5 м JAFAR",95.156


In [106]:
results.to_csv('./results.csv', sep=';', encoding='utf-8-sig')

In [107]:
from IPython.display import FileLink
FileLink(r'results.csv')

/kaggle/working/results.csv